In [2]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
from google.colab import userdata
import os

os.environ['HF_WRITE_TOKEN'] = userdata.get('HF_WRITE_TOKEN')

!huggingface-cli login --add-to-git-credential --token $HF_WRITE_TOKEN


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
%%capture
!apt-get install chromium-browser
!apt-get install chromium-driver
!apt-get install language-pack-ko
!locale-gen ko_KR.UTF-8
!sudo update-locale LANG=ko_KR.UTF-8 LC_MESSAGES=POSIX
!pip install \
    crewai[tools] \
    duckduckgo-search \
    langchain \
    langchain-community \
    arxiv \
    xmltodict \
    langchain-huggingface \
    youtube-transcript-api \
    pytube \
    pyautogen>=0.2.29 \
    hfautogen

## Crewai task procedure

In [ ]:
%%writefile setttings.yml
#@title searx setting.yml
general:
    debug : False # Debug mode, only for development
    instance_name : "searx" # displayed name
    contact_url: False # mailto:contact@example.com
    enable_stats: False # activate /stats page - note: it may leak usage data

brand:
    git_url: https://github.com/searx/searx
    git_branch: master
    issue_url: https://github.com/searx/searx/issues
    docs_url: https://searx.github.io/searx
    public_instances: https://searx.space
    wiki_url: https://github.com/searx/searx/wiki
    twitter_url: https://twitter.com/Searx_engine

search:
    safe_search : 0 # Filter results. 0: None, 1: Moderate, 2: Strict
    autocomplete : "" # Existing autocomplete backends: "dbpedia", "duckduckgo", "google", "startpage", "swisscows", "qwant", "wikipedia" - leave blank to turn it off by default
    default_lang : "" # Default search language - leave blank to detect from browser information or use codes from 'languages.py'
    ban_time_on_fail : 5 # ban time in seconds after engine errors
    max_ban_time_on_fail : 120 # max ban time in seconds after engine errors
    prefer_configured_language: False # increase weight of results in configured language in ranking

server:
    port : 8888
    bind_address : "127.0.0.1" # address to listen on
    secret_key : "d2dbc0d86f3eaf199aa8c38fae36e1fe89f8fdd5" # change this!
    base_url : False # Set custom base_url. Possible values: False or "https://your.custom.host/location/"
    image_proxy : False # Proxying image results through searx
    http_protocol_version : "1.0"  # 1.0 and 1.1 are supported
    method: "POST" # POST queries are more secure as they don't show up in history but may cause problems when using Firefox containers
    default_http_headers:
        X-Content-Type-Options : nosniff
        X-XSS-Protection : 1; mode=block
        X-Download-Options : noopen
        X-Robots-Tag : noindex, nofollow
        Referrer-Policy : no-referrer

ui:
    autofocus : True # Autofocus search input
    archive_today : False # show archive.today links
    static_path : "" # Custom static path - leave it blank if you didn't change
    templates_path : "" # Custom templates path - leave it blank if you didn't change
    default_theme : oscar # ui theme
    default_locale : "" # Default interface locale - leave blank to detect from browser information or use codes from the 'locales' config section
    theme_args :
        oscar_style : logicodev # default style of oscar
#   results_on_new_tab: False  # Open result links in a new tab by default
#   categories_order :
#     - general
#     - files
#     - map
#     - it
#     - science

# Lock arbitrary settings on the preferences page.
# To find the ID of the user setting you want to lock, check
# the ID of the form on the page "preferences".
#preferences:
#    lock:
#      - language
#      - autocomplete
#      - method

# searx supports result proxification using an external service: https://github.com/asciimoo/morty
# uncomment below section if you have running morty proxy
# the key is base64 encoded (keep the !!binary notation)
# Note: since commit af77ec3, morty accepts a base64 encoded key.
#result_proxy:
#    url : http://127.0.0.1:3000/
#    key : !!binary "your_morty_proxy_key"

outgoing: # communication with search engines
    request_timeout : 2.0 # default timeout in seconds, can be override by engine
    # max_request_timeout: 10.0 # the maximum timeout in seconds
    useragent_suffix : "" # suffix of searx_useragent, could contain information like an email address to the administrator
    pool_connections : 100 # Number of different hosts
    pool_maxsize : 10 # Number of simultaneous requests by host
# uncomment below section if you want to use a proxy
# see https://2.python-requests.org/en/latest/user/advanced/#proxies
# SOCKS proxies are also supported: see https://2.python-requests.org/en/latest/user/advanced/#socks
#    proxies:
#        http:
#            - http://proxy1:8080
#            - http://proxy2:8080
#        https:
#            - http://proxy1:8080
#            - http://proxy2:8080
#    using_tor_proxy : True
#    extra_proxy_timeout : 10.0 # Extra seconds to add in order to account for the time taken by the proxy
# uncomment below section only if you have more than one network interface
# which can be the source of outgoing search requests
#    source_ips:
#        - 1.1.1.1
#        - 1.1.1.2

# External plugin configuration
# See https://searx.github.io/searx/dev/plugins.html for more details
#
# plugins:
#   - plugin1
#   - plugin2
#   - ...

# uncomment below section if you want to configure which plugin is enabled by default
#
# enabled_plugins:
#   - "HTTPS rewrite"
#   - ...

# Example to rewrite hostnames in external links
#
# enabled_plugins:
#   - 'Hostname replace'
# hostname_replace:
#   '(.*\.)?youtube\.com$':           'invidious.example.com'
#   '(.*\.)?youtu\.be$':              'invidious.example.com'
#   '(.*\.)?youtube-noocookie\.com$': 'yotter.example.com'
#   '(.*\.)?reddit\.com$':            'teddit.example.com'
#   '(.*\.)?redd\.it$':               'teddit.example.com'
#   '(www\.)?twitter\.com$':          'nitter.example.com'

checker:
    # disable checker when in debug mode
    off_when_debug: True

    # scheduling: interval or int
    # use "scheduling: False" to disable scheduling
    # to activate the scheduler:
    # * uncomment "scheduling" section
    # * add "cache2 = name=searxcache,items=2000,blocks=2000,blocksize=4096,bitmap=1" to your uwsgi.ini

    # scheduling:
    #    start_after: [300, 1800]  # delay to start the first run of the checker
    #    every: [86400, 90000]  # how often the checker runs

    # additional tests: only for the YAML anchors (see the engines section)

    additional_tests:
      rosebud: &test_rosebud
        matrix:
          query: rosebud
          lang: en
        result_container:
          - not_empty
          - ['one_title_contains', 'citizen kane']
        test:
          - unique_results

      android: &test_android
        matrix:
          query: ['android']
          lang: ['en', 'de', 'fr', 'zh-CN']
        result_container:
          - not_empty
          - ['one_title_contains', 'google']
        test:
          - unique_results

    # tests: only for the YAML anchors (see the engines section)
    tests:
      infobox: &tests_infobox
        infobox:
          matrix:
            query: ["linux", "new york", "bbc"]
          result_container:
            - has_infobox

engines:
  - name: apk mirror
    engine: apkmirror
    timeout: 4.0
    shortcut: apkm
    disabled: True

# Requires Tor
  - name : ahmia
    engine : ahmia
    categories : onions
    enable_http : True
    shortcut : ah

  - name : arch linux wiki
    engine : archlinux
    shortcut : al

  - name : archive is
    engine : xpath
    search_url : https://archive.is/search/?q={query}
    url_xpath : (//div[@class="TEXT-BLOCK"]/a)/@href
    title_xpath : (//div[@class="TEXT-BLOCK"]/a)
    content_xpath : //div[@class="TEXT-BLOCK"]/ul/li
    categories : general
    timeout : 7.0
    disabled : True
    shortcut : ai
    soft_max_redirects: 1
    about:
      website: https://archive.is/
      wikidata_id: Q13515725
      official_api_documentation: https://mementoweb.org/depot/native/archiveis/
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : arxiv
    engine : arxiv
    shortcut : arx
    categories : science
    timeout : 4.0

#  tmp suspended:  dh key too small
#  - name : base
#    engine : base
#    shortcut : bs

  - name: bandcamp
    engine: bandcamp
    shortcut: bc
    categories: music

  - name : wikipedia
    engine : wikipedia
    shortcut : wp
    base_url : 'https://{language}.wikipedia.org/'

    #The fulltext and title parameter is necessary for Wikimini because sometimes it will not show the results and redirect instead
  - name: wikimini
    engine: xpath
    shortcut: wkmn
    search_url : https://fr.wikimini.org/w/index.php?search={query}&title=Sp%C3%A9cial%3ASearch&fulltext=Search
    url_xpath : //li/div[@class="mw-search-result-heading"]/a/@href
    title_xpath : //li//div[@class="mw-search-result-heading"]/a
    content_xpath : //li/div[@class="searchresult"]
    categories : general
    disabled : True
    about:
      website: https://wikimini.org/
      wikidata_id: Q3568032
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : bing
    engine : bing
    shortcut : bi

  - name : bing images
    engine : bing_images
    shortcut : bii

  - name : bing news
    engine : bing_news
    shortcut : bin

  - name : bing videos
    engine : bing_videos
    shortcut : biv

  - name : bitbucket
    engine : xpath
    paging : True
    search_url : https://bitbucket.org/repo/all/{pageno}?name={query}
    url_xpath : //article[@class="repo-summary"]//a[@class="repo-link"]/@href
    title_xpath : //article[@class="repo-summary"]//a[@class="repo-link"]
    content_xpath : //article[@class="repo-summary"]/p
    categories : it
    timeout : 4.0
    disabled : True
    shortcut : bb
    about:
      website: https://bitbucket.org/
      wikidata_id: Q2493781
      official_api_documentation: https://developer.atlassian.com/bitbucket
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : btdigg
    engine : btdigg
    shortcut : bt

  - name : ccc-tv
    engine : xpath
    paging : False
    search_url : https://media.ccc.de/search/?q={query}
    url_xpath : //div[@class="caption"]/h3/a/@href
    title_xpath : //div[@class="caption"]/h3/a/text()
    content_xpath : //div[@class="caption"]/h4/@title
    categories : videos
    disabled : True
    shortcut : c3tv
    about:
      website: https://media.ccc.de/
      wikidata_id: Q80729951
      official_api_documentation: https://github.com/voc/voctoweb
      use_official_api: false
      require_api_key: false
      results: HTML

  - name: openverse
    engine: openverse
    categories: images
    shortcut: opv

# - name : core.ac.uk
#   engine : core
#   categories : science
#   shortcut : cor
#   # get your API key from: https://core.ac.uk/api-keys/register/
#   api_key : 'unset'

  - name : crossref
    engine : json_engine
    paging : True
    search_url : https://search.crossref.org/dois?q={query}&page={pageno}
    url_query : doi
    title_query : title
    title_html_to_text: True
    content_query : fullCitation
    content_html_to_text: True
    categories : science
    shortcut : cr
    about:
      website: https://www.crossref.org/
      wikidata_id: Q5188229
      official_api_documentation: https://github.com/CrossRef/rest-api-doc
      use_official_api: false
      require_api_key: false
      results: JSON

  - name : currency
    engine : currency_convert
    categories : general
    shortcut : cc

  - name : deezer
    engine : deezer
    shortcut : dz

  - name : deviantart
    engine : deviantart
    shortcut : da
    timeout : 3.0

  - name : ddg definitions
    engine : duckduckgo_definitions
    shortcut : ddd
    weight : 2
    disabled : True
    tests: *tests_infobox

# cloudflare protected
#  - name : digbt
#    engine : digbt
#    shortcut : dbt
#    timeout : 6.0
#    disabled : True

  - name : digg
    engine : digg
    shortcut : dg

  - name : erowid
    engine : xpath
    paging : True
    first_page_num : 0
    page_size : 30
    search_url : https://www.erowid.org/search.php?q={query}&s={pageno}
    url_xpath : //dl[@class="results-list"]/dt[@class="result-title"]/a/@href
    title_xpath : //dl[@class="results-list"]/dt[@class="result-title"]/a/text()
    content_xpath : //dl[@class="results-list"]/dd[@class="result-details"]
    categories : general
    shortcut : ew
    disabled : True
    about:
      website: https://www.erowid.org/
      wikidata_id: Q1430691
      official_api_documentation:
      use_official_api: false
      require_api_key: false
      results: HTML

  - name: emojipedia
    engine: emojipedia
    timeout: 4.0
    shortcut: em
    disabled: True

#  - name : elasticsearch
#    shortcut : es
#    engine : elasticsearch
#    base_url : http://localhost:9200
#    username : elastic
#    password : changeme
#    index : my-index
#    # available options: match, simple_query_string, term, terms, custom
#    query_type : match
#    # if query_type is set to custom, provide your query here
#    #custom_query_json: {"query":{"match_all": {}}}
#    #show_metadata: False
#    disabled : True

  - name : wikidata
    engine : wikidata
    shortcut : wd
    timeout : 3.0
    weight : 2
    tests: *tests_infobox

  - name : duckduckgo
    engine : duckduckgo
    shortcut : ddg
    disabled : True

  - name : duckduckgo images
    engine : duckduckgo_images
    shortcut : ddi
    timeout: 3.0
    disabled : True

  - name : tineye
    engine : tineye
    shortcut : tin
    paging : True
    timeout : 9.0
    disabled : True

  - name : etools
    engine : etools
    shortcut : eto
    disabled : True
    additional_tests:
      rosebud: *test_rosebud

  - name : etymonline
    engine : xpath
    paging : True
    search_url : https://etymonline.com/search?page={pageno}&q={query}
    url_xpath : //a[contains(@class, "word__name--")]/@href
    title_xpath : //a[contains(@class, "word__name--")]
    content_xpath : //section[contains(@class, "word__defination")]
    first_page_num : 1
    shortcut : et
    disabled : True
    about:
      website: https://www.etymonline.com/
      wikidata_id: Q1188617
      official_api_documentation:
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : ebay
    engine : ebay
    shortcut : eb
    disabled : True
    timeout: 5

  - name : 1x
    engine : www1x
    shortcut : 1x
    timeout : 3.0
    disabled : True

  - name : fdroid
    engine : fdroid
    shortcut : fd
    disabled : True

  - name : flickr
    categories : images
    shortcut : fl
# You can use the engine using the official stable API, but you need an API key
# See : https://www.flickr.com/services/apps/create/
#    engine : flickr
#    api_key: 'apikey' # required!
# Or you can use the html non-stable engine, activated by default
    engine : flickr_noapi

  - name : free software directory
    engine : mediawiki
    shortcut : fsd
    categories : it
    base_url : https://directory.fsf.org/
    number_of_results : 5
# what part of a page matches the query string: title, text, nearmatch
# title - query matches title, text - query matches the text of page, nearmatch - nearmatch in title
    search_type : title
    timeout : 5.0
    disabled : True
    about:
      website: https://directory.fsf.org/
      wikidata_id: Q2470288

  # - name : freesound
    # engine : freesound
    # shortcut : fnd
    # disabled : True
    # timeout : 15.0
    # api_key : MyAPIkey
    # API key required, see : https://freesound.org/docs/api/overview.html

  - name : frinkiac
    engine : frinkiac
    shortcut : frk
    disabled : True

  - name : genius
    engine : genius
    shortcut : gen

  - name : gigablast
    engine : gigablast
    shortcut : gb
    timeout : 3.0
    # Collection names to search in, separated by whitespaces.
    # By default searx, queries the main collection.
    #collections: main
    # Search types, not required. possible values: images, news
    #search_type: images
    # Enable fast to get results quicker.
    #fast : 1
    disabled: True
    additional_tests:
      rosebud: *test_rosebud

  - name : gentoo
    engine : gentoo
    shortcut : ge

  - name : gitlab
    engine : json_engine
    paging : True
    search_url : https://gitlab.com/api/v4/projects?search={query}&page={pageno}
    url_query : web_url
    title_query : name_with_namespace
    content_query : description
    page_size : 20
    categories : it
    shortcut : gl
    timeout : 10.0
    disabled : True
    about:
      website: https://about.gitlab.com/
      wikidata_id: Q16639197
      official_api_documentation: https://docs.gitlab.com/ee/api/
      use_official_api: false
      require_api_key: false
      results: JSON

  - name : github
    engine : github
    shortcut : gh

    # This a Gitea service. If you would like to use a different instance,
    # change codeberg.org to URL of the desired Gitea host. Or you can create
    # a new engine by copying this and changing the name, shortcut and search_url.
  - name : codeberg
    engine : json_engine
    search_url : https://codeberg.org/api/v1/repos/search?q={query}&limit=10
    url_query : html_url
    title_query : name
    content_query : description
    categories : it
    shortcut : cb
    disabled : True
    about:
      website: https://codeberg.org/
      wikidata_id:
      official_api_documentation: https://try.gitea.io/api/swagger
      use_official_api: false
      require_api_key: false
      results: JSON

  - name : google
    engine : google
    shortcut : go
    use_mobile_ui: false
    # additional_tests:
    #   android: *test_android

  - name : google images
    engine : google_images
    shortcut : goi
    # additional_tests:
    #   android: *test_android
    #   dali:
    #     matrix:
    #       query: ['Dali Christ']
    #       lang: ['en', 'de', 'fr', 'zh-CN']
    #     result_container:
    #       - ['one_title_contains', 'Salvador']

  - name : google news
    engine : google_news
    shortcut : gon
    # additional_tests:
    #   android: *test_android

  - name : google videos
    engine : google_videos
    shortcut : gov
    # additional_tests:
    #   android: *test_android

  - name : google scholar
    engine : google_scholar
    shortcut : gos

  - name : google play apps
    engine : xpath
    search_url : https://play.google.com/store/search?q={query}&c=apps
    results_xpath : '//div[@class="ImZGtf mpg5gc"]'
    title_xpath : './/div[@class="RZEgze"]//div[@class="kCSSQe"]//a'
    url_xpath : './/div[@class="RZEgze"]//div[@class="kCSSQe"]//a/@href'
    content_xpath : './/div[@class="RZEgze"]//a[@class="mnKHRc"]'
    thumbnail_xpath : './/div[@class="uzcko"]/div/span[1]//img/@data-src'
    categories : files
    shortcut : gpa
    disabled : True
    about:
      website: https://play.google.com/
      wikidata_id: Q79576
      official_api_documentation:
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : gpodder
    engine : json_engine
    shortcut : gpod
    timeout : 4.0
    paging : False
    search_url : https://gpodder.net/search.json?q={query}
    url_query : url
    title_query : title
    content_query : description
    page_size : 19
    categories : social media, files, general
    disabled: True
    about:
      website: https://gpodder.net
      wikidata_id: Q3093354
      official_api_documentation: https://gpoddernet.readthedocs.io/en/latest/api/
      use_official_api: false
      requires_api_key: false
      results: JSON

  - name : google play movies
    engine : xpath
    search_url : https://play.google.com/store/search?q={query}&c=movies
    results_xpath : '//div[@class="ImZGtf mpg5gc"]'
    title_xpath : './/div[@class="RZEgze"]//div[@class="kCSSQe"]//a'
    url_xpath : './/div[@class="RZEgze"]//div[@class="kCSSQe"]//a/@href'
    content_xpath : './/div[@class="kCSSQe"]'
    thumbnail_xpath : './/div[@class="uzcko"]/div/span[1]//img/@data-src'
    categories : videos
    shortcut : gpm
    disabled : True
    about:
      website: https://play.google.com/
      wikidata_id: Q79576
      official_api_documentation:
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : geektimes
    engine : xpath
    paging : True
    search_url : https://habr.com/en/search/page{pageno}/?q={query}
    url_xpath : //article[contains(@class, "post")]//a[@class="post__title_link"]/@href
    title_xpath : //article[contains(@class, "post")]//a[@class="post__title_link"]
    content_xpath : //article[contains(@class, "post")]//div[contains(@class, "post__text")]
    categories : it
    timeout : 4.0
    disabled : True
    shortcut : gt
    about:
      website: https://geektimes.ru/
      wikidata_id: Q50572423
      official_api_documentation:
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : habrahabr
    engine : xpath
    paging : True
    search_url : https://habrahabr.ru/search/page{pageno}/?q={query}
    url_xpath : //article[contains(@class, "post")]//a[@class="post__title_link"]/@href
    title_xpath : //article[contains(@class, "post")]//a[@class="post__title_link"]
    content_xpath : //article[contains(@class, "post")]//div[contains(@class, "post__text")]
    categories : it
    timeout : 4.0
    disabled : True
    shortcut : habr
    about:
      website: https://habr.com/
      wikidata_id: Q4494434
      official_api_documentation: https://habr.com/en/docs/help/api/
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : hoogle
    engine : xpath
    search_url : https://hoogle.haskell.org/?hoogle={query}&start={pageno}
    results_xpath : '//div[@class="result"]'
    title_xpath : './div[@class="ans"]'
    url_xpath : './div[@class="ans"]//a/@href'
    content_xpath : './div[contains(@class, "doc")]'
    categories : it
    shortcut : ho
    about:
      website: https://hoogle.haskell.org/
      wikidata_id: Q34010
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : imdb
    engine : imdb
    shortcut : imdb
    timeout : 6.0
    disabled : True

  - name : ina
    engine : ina
    shortcut : in
    timeout : 6.0
    disabled : True

  - name : invidious
    engine : invidious
    base_url :
      - https://invidious.tube/
      - https://invidious.snopyta.org/
    shortcut: iv
    timeout : 5.0
    disabled : True

  - name: ipfs search
    engine: ipfs_search
    shortcut: ipfs
    paging: True
    timeout: 5.0
    disabled: True

  - name: kickass
    engine : kickass
    shortcut : kc
    timeout : 4.0
    disabled : True

  - name : library genesis
    engine : xpath
    paging : True
    page_size : 25
    search_url : https://libgen.rs/search.php?req={query}&page={pageno}
    url_xpath : //a[contains(@href,"library.lol/main")]/@href
    title_xpath : //a[contains(@href,"book/")]/text()[1]
    content_xpath : //td/a[1][contains(@href,"=author")]/text()
    categories : general, files
    timeout : 7.0
    disabled : True
    shortcut : lg
    about:
      website: https://libgen.rs/
      wikidata_id: Q22017206
      official_api_documentation:
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : azlyrics
    shortcut : azl
    engine : xpath
    timeout : 4.0
    disabled : True
    categories : general
    paging : True
    search_url : https://search.azlyrics.com/search.php?q={query}&w=lyrics&p={pageno}
    url_xpath : //td[@class="text-left visitedlyr"]/a/@href
    title_xpath : //span/b/text()
    content_xpath : //td[@class="text-left visitedlyr"]/a/small
    about:
      website: https://azlyrics.com
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : library of congress
    engine : loc
    shortcut : loc
    categories : images

  - name: lingva
    engine: lingva
    shortcut: lv
    # set lingva instance in url, by default it will use the official instance
    # url: https://lingva.ml

  - name: lobste.rs
    engine: xpath
    search_url: https://lobste.rs/search?utf8=%E2%9C%93&q={query}&what=stories&order=relevance
    results_xpath: //li[contains(@class, "story")]
    url_xpath: .//a[@class="u-url"]/@href
    title_xpath: .//a[@class="u-url"]
    content_xpath: .//a[@class="domain"]
    categories: it
    shortcut: lo
    timeout: 5.0
    disabled: true
    about:
      website: https://lobste.rs/
      wikidata_id: Q60762874
      official_api_documentation:
      use_official_api: false
      require_api_key: false
      results: HTML

#  - name : meilisearch
#    engine : meilisearch
#    shortcut: mes
#    enable_http: True
#    base_url : http://localhost:7700
#    index : my-index

  - name : microsoft academic
    engine : microsoft_academic
    categories : science
    shortcut : ma

  - name : mixcloud
    engine : mixcloud
    shortcut : mc

# MongoDB engine
# Required dependency: pymongo
#  - name : mymongo
#    engine : mongodb
#    shortcut : md
#    exact_match_only : False
#    host : '127.0.0.1'
#    port : 27017
#    results_per_page : 20
#    database : 'business'
#    collection : 'reviews' # name of the db collection
#    key : 'name' # key in the collection to search for

  - name : npm
    engine : json_engine
    paging : True
    search_url : https://api.npms.io/v2/search?q={query}&size=25&from={pageno}
    results_query : results
    url_query : package/links/npm
    title_query : package/name
    content_query : package/description
    page_size : 25
    categories : it
    disabled: True
    timeout: 5.0
    shortcut : npm
    about:
      website: https://npms.io/
      wikidata_id: Q7067518
      official_api_documentation: https://api-docs.npms.io/
      use_official_api: false
      require_api_key: false
      results: JSON

  - name : nyaa
    engine : nyaa
    shortcut : nt
    disabled : True

  - name : mankier
    engine : json_engine
    search_url : https://www.mankier.com/api/v2/mans/?q={query}
    results_query : results
    url_query : url
    title_query : name
    content_query : description
    categories : it
    shortcut : man
    about:
      website: https://www.mankier.com/
      official_api_documentation: https://www.mankier.com/api
      use_official_api: true
      require_api_key: false
      results: JSON

  - name : openairedatasets
    engine : json_engine
    paging : True
    search_url : https://api.openaire.eu/search/datasets?format=json&page={pageno}&size=10&title={query}
    results_query : response/results/result
    url_query : metadata/oaf:entity/oaf:result/children/instance/webresource/url/$
    title_query : metadata/oaf:entity/oaf:result/title/$
    content_query : metadata/oaf:entity/oaf:result/description/$
    content_html_to_text: True
    categories : science
    shortcut : oad
    timeout: 5.0
    about:
      website: https://www.openaire.eu/
      wikidata_id: Q25106053
      official_api_documentation: https://api.openaire.eu/
      use_official_api: false
      require_api_key: false
      results: JSON

  - name : openairepublications
    engine : json_engine
    paging : True
    search_url : https://api.openaire.eu/search/publications?format=json&page={pageno}&size=10&title={query}
    results_query : response/results/result
    url_query : metadata/oaf:entity/oaf:result/children/instance/webresource/url/$
    title_query : metadata/oaf:entity/oaf:result/title/$
    content_query : metadata/oaf:entity/oaf:result/description/$
    content_html_to_text: True
    categories : science
    shortcut : oap
    timeout: 5.0
    about:
      website: https://www.openaire.eu/
      wikidata_id: Q25106053
      official_api_documentation: https://api.openaire.eu/
      use_official_api: false
      require_api_key: false
      results: JSON

#  - name : opensemanticsearch
#    engine : opensemantic
#    shortcut : oss
#    base_url : 'http://localhost:8983/solr/opensemanticsearch/'

  - name : openstreetmap
    engine : openstreetmap
    shortcut : osm

#  - name : prowlarr
#    engine : prowlarr
#    shortcut : prow
#    categories : files
#    enable_http : True
#    api_key : ''
#    indexer_ids : 5 # comma separated list of indexer ids
#    search_categories : '' # comma separated list of categories
#    search_type : search
#    base_url: http://localhost:9696/api/v1/search?
#    timeout : 50.0
#    disabled : True,
#    about :
#      website : https://wiki.servarr.com/prowlarr
#      wikidata_id : None
#      official_api_documentation : https://wiki.servarr.com/prowlarr/search
#      use_official_api : true
#      require_api_key : true
#      results : JSON

  - name : openrepos
    engine : xpath
    paging : True
    search_url : https://openrepos.net/search/node/{query}?page={pageno}
    url_xpath : //li[@class="search-result"]//h3[@class="title"]/a/@href
    title_xpath : //li[@class="search-result"]//h3[@class="title"]/a
    content_xpath : //li[@class="search-result"]//div[@class="search-snippet-info"]//p[@class="search-snippet"]
    categories : files
    timeout : 4.0
    disabled : True
    shortcut : or
    about:
      website: https://openrepos.net/
      wikidata_id:
      official_api_documentation:
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : pdbe
    engine : pdbe
    shortcut : pdb
# Hide obsolete PDB entries.
# Default is not to hide obsolete structures
#    hide_obsolete : False

  - name : photon
    engine : photon
    shortcut : ph

  - name : piratebay
    engine : piratebay
    shortcut : tpb
    # You may need to change this URL to a proxy if piratebay is blocked in your country
    url: https://thepiratebay.org/
    timeout : 3.0

# Required dependency: psychopg2
#  - name : postgresql
#    engine : postgresql
#    database : postgres
#    username : postgres
#    password : postgres
#    limit : 10
#    query_str : 'SELECT * from my_table WHERE my_column = %(query)s'
#    shortcut : psql

  - name : pubmed
    engine : pubmed
    shortcut : pub
    categories: science
    timeout : 3.0

  - name : pypi
    shortcut: pypi
    engine: xpath
    paging : True
    search_url : https://pypi.org/search?q={query}&page={pageno}
    results_xpath: /html/body/main/div/div/div/form/div/ul/li/a[@class="package-snippet"]
    url_xpath : ./@href
    title_xpath : ./h3/span[@class="package-snippet__name"]
    content_xpath : ./p
    suggestion_xpath : /html/body/main/div/div/div/form/div/div[@class="callout-block"]/p/span/a[@class="link"]
    first_page_num : 1
    categories: it
    about:
      website: https://pypi.org
      wikidata_id: Q2984686
      official_api_documentation: https://warehouse.readthedocs.io/api-reference/index.html
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : qwant
    engine : qwant
    shortcut : qw
    categories : general
    disabled : True
    additional_tests:
      rosebud: *test_rosebud

  - name : qwant images
    engine : qwant
    shortcut : qwi
    disabled: True
    categories : images

  - name : qwant news
    engine : qwant
    shortcut : qwn
    categories : news

  - name: qwant videos
    engine: qwant
    shortcut: qwv
    categories: videos
    disabled: True
    network: qwant

#  - name: library
#    engine: recoll
#    shortcut: lib
#    base_url: 'https://recoll.example.org/'
#    search_dir: ''
#    mount_prefix: /export
#    dl_prefix: 'https://download.example.org'
#    timeout: 30.0
#    categories: files
#    disabled: True

#  - name: recoll library reference
#    engine: recoll
#    base_url: 'https://recoll.example.org/'
#    search_dir: reference
#    mount_prefix: /export
#    dl_prefix: 'https://download.example.org'
#    shortcut: libr
#    timeout: 30.0
#    categories: files
#    disabled: True

  - name : reddit
    engine : reddit
    shortcut : re
    page_size : 25
    timeout : 10.0
    disabled : True

# Required dependency: redis
#  - name : myredis
#    engine : redis_server
#    exact_match_only : False
#    host : '127.0.0.1'
#    port : 6379
#    password : ''
#    db : 0

# tmp suspended: bad certificate
#  - name : scanr structures
#    shortcut: scs
#    engine : scanr_structures
#    disabled : True

  - name: sepiasearch
    engine: sepiasearch
    shortcut: sep

  - name : soundcloud
    engine : soundcloud
    shortcut : sc

  - name: stackoverflow
    engine: stackexchange
    shortcut: st
    api_site: 'stackoverflow'
    categories: it

  - name: askubuntu
    engine: stackexchange
    shortcut: ubuntu
    api_site: 'askubuntu'
    categories: it

  - name: superuser
    engine: stackexchange
    shortcut: su
    api_site: 'superuser'
    categories: it

  - name : searchcode code
    engine : searchcode_code
    shortcut : scc
    disabled : True

  - name : framalibre
    engine : framalibre
    shortcut : frl
    disabled : True

#  - name : searx
#    engine : searx_engine
#    shortcut : se
#    instance_urls :
#        - http://127.0.0.1:8888/
#        - ...
#    disabled : True

  - name : semantic scholar
    engine : semantic_scholar
    disabled : True
    shortcut : se
    categories : science

# Spotify needs API credentials
#  - name : spotify
#    engine : spotify
#    shortcut : stf
#    api_client_id : *******
#    api_client_secret : *******

#  - name : solr
#    engine : solr
#    shortcut : slr
#    base_url : http://localhost:8983
#    collection : collection_name
#    sort : '' # sorting: asc or desc
#    field_list : '' # comma separated list of field names to display on the UI
#    default_fields : '' # default field to query
#    query_fields : '' # query fields
#    enable_http : True

# - name : springer nature
#   engine : springer
#   # get your API key from: https://dev.springernature.com/signup
#   api_key : 'unset' # working API key, for test & debug: "a69685087d07eca9f13db62f65b8f601"
#   disabled: True
#   shortcut : springer
#   categories : science
#   timeout : 6.0

  - name : startpage
    engine : startpage
    shortcut : sp
    timeout : 6.0
    disabled : True
    additional_tests:
      rosebud: *test_rosebud

  - name : tokyotoshokan
    engine : tokyotoshokan
    shortcut : tt
    timeout : 6.0
    disabled : True

  - name : solidtorrents
    engine : solidtorrents
    shortcut : solid
    timeout : 3.0
    disabled : True

#  - name : sqlite
#    engine : sqlite
#    shortcut: sq
#    database : mydb
#    query_str : 'SELECT * FROM mytable WHERE fieldname=:query'
#    disabled : True

  - name : torrentz
    engine : torrentz
    shortcut : tor
    url: https://torrentz2.eu/
    timeout : 3.0

# Requires Tor
  - name : torch
    engine : xpath
    paging : True
    search_url : http://xmh57jrknzkhv6y3ls3ubitzfqnkrwxhopf5aygthi7d6rplyvk3noyd.onion/cgi-bin/omega/omega?P={query}&DEFAULTOP=and
    results_xpath : //table//tr
    url_xpath : ./td[2]/a
    title_xpath : ./td[2]/b
    content_xpath : ./td[2]/small
    categories : onions
    enable_http : True
    shortcut : tch

  - name : tvmaze
    engine : tvmaze
    timeout : 5.0
    shortcut : tm
    disabled : true

# maybe in a fun category
#  - name : uncyclopedia
#    engine : mediawiki
#    shortcut : unc
#    base_url : https://uncyclopedia.wikia.com/
#    number_of_results : 5


  - name : urbandictionary
    engine        : xpath
    search_url    : https://www.urbandictionary.com/define.php?term={query}
    url_xpath     : //*[@class="word"]/@href
    title_xpath   : //*[@class="def-header"]
    content_xpath : //*[@class="meaning"]
    shortcut : ud
    disabled : True

  - name : unsplash
    engine : unsplash
    disabled: True
    shortcut : us

  - name : yahoo
    engine : yahoo
    shortcut : yh
    disabled : True

  - name : yahoo news
    engine : yahoo_news
    shortcut : yhn

  - name : youtube
    shortcut : yt
    # You can use the engine using the official stable API, but you need an API key
    # See : https://console.developers.google.com/project
    #    engine : youtube_api
    #    api_key: 'apikey' # required!
    # Or you can use the html non-stable engine, activated by default
    engine : youtube_noapi

  - name : yggtorrent
    engine : yggtorrent
    shortcut : ygg
    url: https://www4.yggtorrent.li/
    disabled : True
    timeout : 4.0

  - name : dailymotion
    engine : dailymotion
    shortcut : dm

  - name : vimeo
    engine : vimeo
    shortcut : vm

  - name : wiby
    engine : json_engine
    paging : True
    search_url : https://wiby.me/json/?q={query}&p={pageno}
    url_query : URL
    title_query : Title
    content_query : Snippet
    categories : general
    shortcut : wib
    disabled : True

  - name : wikibooks
    engine : mediawiki
    shortcut : wb
    categories : general
    base_url : "https://{language}.wikibooks.org/"
    number_of_results : 5
    search_type : text
    disabled : True
    about:
      website: https://www.wikibooks.org/
      wikidata_id: Q367

  - name : wikinews
    engine : mediawiki
    shortcut : wn
    categories : news
    base_url : "https://{language}.wikinews.org/"
    number_of_results : 5
    search_type : text
    disabled : True
    about:
      website: https://www.wikinews.org/
      wikidata_id: Q964

  - name : wikiquote
    engine : mediawiki
    shortcut : wq
    categories : general
    base_url : "https://{language}.wikiquote.org/"
    number_of_results : 5
    search_type : text
    disabled : True
    additional_tests:
      rosebud: *test_rosebud
    about:
      website: https://www.wikiquote.org/
      wikidata_id: Q369

  - name : wikisource
    engine : mediawiki
    shortcut : ws
    categories : general
    base_url : "https://{language}.wikisource.org/"
    number_of_results : 5
    search_type : text
    disabled : True
    about:
      website: https://www.wikisource.org/
      wikidata_id: Q263

  - name : wiktionary
    engine : mediawiki
    shortcut : wt
    categories : general
    base_url : "https://{language}.wiktionary.org/"
    number_of_results : 5
    search_type : text
    disabled : True
    about:
      website: https://www.wiktionary.org/
      wikidata_id: Q151

  - name : wikiversity
    engine : mediawiki
    shortcut : wv
    categories : general
    base_url : "https://{language}.wikiversity.org/"
    number_of_results : 5
    search_type : text
    disabled : True
    about:
      website: https://www.wikiversity.org/
      wikidata_id: Q370

  - name : wikivoyage
    engine : mediawiki
    shortcut : wy
    categories : general
    base_url : "https://{language}.wikivoyage.org/"
    number_of_results : 5
    search_type : text
    disabled : True
    about:
      website: https://www.wikivoyage.org/
      wikidata_id: Q373

  - name : wolframalpha
    shortcut : wa
    # You can use the engine using the official stable API, but you need an API key
    # See : https://products.wolframalpha.com/api/
    # engine : wolframalpha_api
    # api_key: '' # required!
    engine : wolframalpha_noapi
    timeout: 6.0
    categories : science

  - name : dictzone
    engine : dictzone
    shortcut : dc

  - name : mymemory translated
    engine : translated
    shortcut : tl
    timeout : 5.0
    disabled : True
    # You can use without an API key, but you are limited to 1000 words/day
    # See : https://mymemory.translated.net/doc/usagelimits.php
    # api_key : ''

# Required dependency: mysql-connector-python
#  - name : mysql
#    engine : mysql_server
#    database : mydatabase
#    username : user
#    password : pass
#    limit : 10
#    query_str : 'SELECT * from mytable WHERE fieldname=%(query)s'
#    shortcut : mysql

  - name : 1337x
    engine : 1337x
    shortcut : 1337x
    disabled : True

  - name : duden
    engine : duden
    shortcut : du
    disabled : True

  - name : seznam
    shortcut: szn
    engine: seznam
    disabled : True

  - name : mojeek
    shortcut: mjk
    engine: xpath
    paging : True
    search_url : https://www.mojeek.com/search?q={query}&s={pageno}
    results_xpath: /html/body//div[@class="results"]/ul[@class="results-standard"]/li
    url_xpath : ./h2/a/@href
    title_xpath : ./h2
    content_xpath : ./p[@class="s"]
    suggestion_xpath : /html/body//div[@class="top-info"]/p[@class="top-info spell"]/a
    first_page_num : 0
    page_size : 10
    disabled : True
    about:
      website: https://www.mojeek.com/
      wikidata_id: Q60747299
      official_api_documentation: https://www.mojeek.com/services/api.html/
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : dogpile
    shortcut : dp
    engine : xpath
    paging : True
    # Doesn't work without the 'sc' parameter
    search_url : https://www.dogpile.com/serp?q={query}&page={pageno}&sc=Gp0GqqlRVflv20
    url_xpath : //span[@class="web-bing__url"]
    title_xpath : //a[@class="web-bing__title"]
    content_xpath : //span[@class="web-bing__description"]
    page_size : 9
    categories : general
    timeout : 7.0
    disabled : True
    about:
      website: https://www.dogpile.com
      wikidata_id: Q3595363
      use_official_api: false
      require_api_key: false
      results: html

  - name : whaleslide
    shortcut : whale
    engine : json_engine
    paging : True
    # This API Token is needed to work
    search_url : https://search.whaleslide.com/api/v1/search_v2/web/{pageno}?q={query}&api_token=f8OhUDEYKUICHDnIxEgI7Cb4uYyTBqT4nO8iueNbfTO3devS24yElGqM7nCm
    url_query : url
    title_query : title
    title_html_to_text : True
    content_query : abstract
    content_html_to_text : True
    categories : general
    disabled : True
    about:
      website: https://whaleslide.com/
      wikidata_id: # No Wikidata page
      official_api_documentation: false
      use_official_api: false
      require_api_key: false
      results: JSON

  - name : petal
    shortcut : petal
    engine : xpath
    paging : True
    search_url : https://petalsearch.com/search?query={query}&pn={pageno}
    title_xpath : //span[@class="title-name"]
    url_xpath : //div[@class="card-source"]/text()
    content_xpath : //span[@class="desc-cont"]
    suggestion_xpath : //*[@class="rs-text"]/*[contains(class, rs-text)]
    page_size : 12
    categories : general
    timeout : 4.0
    disabled : True
    about:
      website: https://petalsearch.com
      wikidata_id: #
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : ask
    shortcut : ask
    engine : xpath
    timeout : 3.0
    disabled : True
    categories : general
    paging : True
    page_size : 10
    search_url : https://www.ask.com/web?q={query}&qo=pagination&page={pageno}
    title_xpath : //a[@class="PartialSearchResults-item-title-link result-link"]
    url_xpath : //a[@class="PartialSearchResults-item-title-link result-link"]/@href
    content_xpath : //p[@class="PartialSearchResults-item-abstract"]
    suggestion_xpath : //span[@class="PartialRelatedSearch-item-link-text"]
    about:
      website: https://ask.com
      wikidata_id : Q847564
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : naver
    shortcut: nvr
    engine: xpath
    paging : True
    search_url : https://search.naver.com/search.naver?where=webkr&sm=osp_hty&ie=UTF-8&query={query}&start={pageno}
    url_xpath : //a[@class="link_tit"]/@href
    title_xpath : //a[@class="link_tit"]
    content_xpath : //a[@class="total_dsc"]/div
    first_page_num : 1
    page_size : 10
    disabled : True
    about:
      website: https://www.naver.com/
      wikidata_id: Q485639
      official_api_documentation: https://developers.naver.com/docs/nmt/examples/
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : rubygems
    shortcut: rbg
    engine: xpath
    paging : True
    search_url : https://rubygems.org/search?page={pageno}&query={query}
    results_xpath: /html/body/main/div/a[@class="gems__gem"]
    url_xpath : ./@href
    title_xpath : ./span/h2
    content_xpath : ./span/p
    suggestion_xpath : /html/body/main/div/div[@class="search__suggestions"]/p/a
    first_page_num : 1
    categories: it
    disabled : True
    about:
      website: https://rubygems.org/
      wikidata_id: Q1853420
      official_api_documentation: https://guides.rubygems.org/rubygems-org-api/
      use_official_api: false
      require_api_key: false
      results: HTML

  - name : peertube
    engine: peertube
    shortcut: ptb
    paging : True
    base_url : https://peer.tube/
    categories: videos
    disabled : True

  - name : mediathekviewweb
    engine : mediathekviewweb
    shortcut : mvw

  - name : kaufland
    shortcut : kauf
    engine : xpath
    timeout : 3.0
    paging : True
    search_url : https://www.kaufland.de/item/search/?search_value={query}&page={pageno}
    title_xpath : //div[@class="product__title"]/text()
    url_xpath : //article[@class="product"]//a/@href
    content_xpath : //div[@class="price"]
    #thumbnail_xpath : '//div[@class="product__image-container"]//img/@data-src'
    categories : general
    disabled : True
    about:
      website: https://kaufland.de
      wikidata_id: Q685967
      use_official_api: false
      require_api_key: false
      results: HTML

#  - name : yacy
#    engine : yacy
#    shortcut : ya
#    base_url : 'http://localhost:8090'
#    enable_http: True # required if you aren't using HTTPS for your local yacy instance
#    number_of_results : 5
#    timeout : 3.0

  - name : rumble
    engine : rumble
    shortcut : ru
    base_url : https://rumble.com/
    paging : True
    categories: videos
    disabled : True

  - name: wordnik
    engine: wordnik
    shortcut: def
    base_url: https://www.wordnik.com/
    categories: general
    timeout: 5.0
    disabled: True

  - name : woxikon.de synonyme
    engine : xpath
    shortcut : woxi
    categories : general
    timeout : 5.0
    disabled : True
    search_url : https://synonyme.woxikon.de/synonyme/{query}.php
    url_xpath : //div[@class="upper-synonyms"]/a/@href
    content_xpath : //div[@class="synonyms-list-group"]
    title_xpath : //div[@class="upper-synonyms"]/a
    about:
      website: https://www.woxikon.de/
      wikidata_id: # No Wikidata ID
      use_official_api: false
      require_api_key: false
      results: HTML

  - name: sjp.pwn
    engine: sjp
    shortcut: sjp
    base_url: https://sjp.pwn.pl/
    categories: general
    timeout: 5.0
    disabled: True

  - name: brave
    shortcut: brave
    engine: xpath
    paging: true
    first_page_num: 0
    search_url: https://search.brave.com/search?q={query}&offset={pageno}&spellcheck=1
    url_xpath: //a[@class="result-header"]/@href
    title_xpath: //span[@class="snippet-title"]
    content_xpath: //p[1][@class="snippet-description"]
    suggestion_xpath: //div[@class="text-gray h6"]/a
    categories: general
    about:
      website: https://brave.com/search/
      wikidata_id: Q22906900
      use_official_api: false
      require_api_key: false
      results: HTML

  - name: onesearch
    shortcut: onesearch
    engine: onesearch
    disabled: True
    about:
      website: https://www.onesearch.com/
      wikidata_id: None
      use_official_api: false
      require_api_key: false
      results: HTML

# omnom engine - see https://github.com/asciimoo/omnom for more details
#  - name : omnom
#    engine : omnom
#    paging : True
#    base_url : 'http://your.omnom.host/'
#    enable_http : True
#    categories : general
#    shortcut : om

# Doku engine lets you access to any Doku wiki instance:
# A public one or a private/corporate one.
#  - name : ubuntuwiki
#    engine : doku
#    shortcut : uw
#    base_url : 'https://doc.ubuntu-fr.org'

# Be careful when enabling this engine if you are
# running a public instance. Do not expose any sensitive
# information. You can restrict access by configuring a list
# of access tokens under tokens.
#  - name: git grep
#    engine: command
#    command: ['git', 'grep', '{{QUERY}}']
#    shortcut: gg
#    tokens: []
#    disabled: True
#    delimiter:
#        chars: ':'
#        keys: ['filepath', 'code']

# Be careful when enabling this engine if you are
# running a public instance. Do not expose any sensitive
# information. You can restrict access by configuring a list
# of access tokens under tokens.
#  - name: locate
#    engine: command
#    command: ['locate', '{{QUERY}}']
#    shortcut: loc
#    tokens: []
#    disabled: True
#    delimiter:
#        chars: ' '
#        keys: ['line']

# Be careful when enabling this engine if you are
# running a public instance. Do not expose any sensitive
# information. You can restrict access by configuring a list
# of access tokens under tokens.
#  - name: find
#    engine: command
#    command: ['find', '.', '-name', '{{QUERY}}']
#    query_type: path
#    shortcut: fnd
#    tokens: []
#    disabled: True
#    delimiter:
#        chars: ' '
#        keys: ['line']

# Be careful when enabling this engine if you are
# running a public instance. Do not expose any sensitive
# information. You can restrict access by configuring a list
# of access tokens under tokens.
#  - name: pattern search in files
#    engine: command
#    command: ['fgrep', '{{QUERY}}']
#    shortcut: fgr
#    tokens: []
#    disabled: True
#    delimiter:
#        chars: ' '
#        keys: ['line']

# Be careful when enabling this engine if you are
# running a public instance. Do not expose any sensitive
# information. You can restrict access by configuring a list
# of access tokens under tokens.
#  - name: regex search in files
#    engine: command
#    command: ['grep', '{{QUERY}}']
#    shortcut: gr
#    tokens: []
#    disabled: True
#    delimiter:
#        chars: ' '
#        keys: ['line']

locales:
    en : English
    ar : العَرَبِيَّة (Arabic)
    bg : Български (Bulgarian)
    bo : བོད་སྐད་ (Tibetian)
    ca : Català (Catalan)
    cs : Čeština (Czech)
    cy : Cymraeg (Welsh)
    da : Dansk (Danish)
    de : Deutsch (German)
    el_GR : Ελληνικά (Greek_Greece)
    eo : Esperanto (Esperanto)
    es : Español (Spanish)
    et : Eesti (Estonian)
    eu : Euskara (Basque)
    fa_IR : (fārsī) فارسى (Persian)
    fi : Suomi (Finnish)
    fil : Wikang Filipino (Filipino)
    fr : Français (French)
    gl : Galego (Galician)
    he : עברית (Hebrew)
    hr : Hrvatski (Croatian)
    hu : Magyar (Hungarian)
    ia : Interlingua (Interlingua)
    it : Italiano (Italian)
    ja : 日本語 (Japanese)
    lt : Lietuvių (Lithuanian)
    nl : Nederlands (Dutch)
    nl_BE : Vlaams (Dutch_Belgium)
    oc : Lenga D'òc (Occitan)
    pl : Polski (Polish)
    pt : Português (Portuguese)
    pt_BR : Português (Portuguese_Brazil)
    ro : Română (Romanian)
    ru : Русский (Russian)
    sk : Slovenčina (Slovak)
    sl : Slovenski (Slovene)
    sr : српски (Serbian)
    sv : Svenska (Swedish)
    te : తెలుగు (telugu)
    ta : தமிழ் (Tamil)
    tr : Türkçe (Turkish)
    uk : українська мова (Ukrainian)
    vi : tiếng việt (Vietnamese)
    zh : 中文 (Chinese)
    zh_TW : 國語 (Taiwanese Mandarin)

doi_resolvers :
  oadoi.org : 'https://oadoi.org/'
  doi.org : 'https://doi.org/'
  doai.io  : 'https://dissem.in/'
  sci-hub.se : 'https://sci-hub.se/'
  sci-hub.do : 'https://sci-hub.do/'
  scihubtw.tw : 'https://scihubtw.tw/'
  sci-hub.st : 'https://sci-hub.st/'
  sci-hub.bar : 'https://sci-hub.bar/'
  sci-hub.it.nf : 'https://sci-hub.it.nf/'

default_doi_resolver : 'oadoi.org'

Writing setttings.yml


In [ ]:
!git clone https://github.com/searx/searx searx
!mkdir -p "/usr/local/searx"
!cd "/usr/local/searx"
!git clone --branch "master" --origin "origin" "/content/searx" "searx-src"
!cd searx && utils/searx.sh install all
!sed -i -e "s/debug : False/debug : True/g" "settings.yml"
!searx -i
!cd /usr/local/searx/searx-src
!export SEARX_SETTINGS_PATH="settings.yml"
!pip install babel flask flask-babel langdetect lxml python-dateutil Brotli certifi requests[socks]
!python searx/searx/webapp.py

Cloning into 'searx'...
remote: Enumerating objects: 35513, done.
remote: Counting objects: 100% (5021/5021), done.
remote: Compressing objects: 100% (443/443), done.
remote: Total 35513 (delta 4760), reused 4578 (delta 4578), pack-reused 30492 (from 1)
Receiving objects: 100% (35513/35513), 23.03 MiB | 26.00 MiB/s, done.
Resolving deltas: 100% (24141/24141), done.
Cloning into 'searx-src'...
done.

searx@fb9bb0cc6a10 

Install searx@fb9bb0cc6a10  (service)

installation of packages
------------------------

package(s)::

   python3-dev python3-babel python3-venv uwsgi uwsgi-plugin-python3 git
   build-essential libxslt-dev zlib1g-dev libffi-dev libssl-dev shellcheck

Should packages be installed? [YES/no] Yes
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:4 https://r2u.stat.illino

In [ ]:
from langchain_community.utilities import SearxSearchWrapper

"https://search.bus-hit.me/"
"https://www.gruble.de"
"https://www.bing.com"
"https://www.google.com"

search = SearxSearchWrapper(searx_host="https://localhost:8888", k=1)

search.run("What is the capital of France")


ConnectionError: HTTPSConnectionPool(host='localhost', port=8888): Max retries exceeded with url: /?language=en&format=json&q=What+is+the+capital+of+France (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7b63bb5efd00>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
import os
import datetime

from crewai import Agent, Task, Crew, Process, Pipeline
from crewai_tools import (SerperDevTool, ScrapeWebsiteTool, DallETool,
                          WebsiteSearchTool, SeleniumScrapingTool, tool)
from crewai.tasks.task_output import TaskOutput
from crewai.tasks.conditional_task import ConditionalTask

from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.tools import (WikipediaQueryRun,
                                       PubmedQueryRun, YouTubeSearchTool, OpenWeatherMapQueryRun)
from langchain_community.utilities import SearxSearchWrapper
from langchain_community.tools import DuckDuckGoSearchRun, SearxSearchRun
from langchain_community.agent_toolkits.jira.toolkit import JiraToolkit
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities import SearxSearchWrapper
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.google_trends.tool import GoogleTrendsQueryRun
from langchain.llms import Ollama

from pydantic import BaseModel
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")

class DPODataForm(BaseModel):
    prompt:str
    chosen:str
    rejected:str

class AgentSharedForm(BaseModel):
    prompt:str
    chosen:str
    rejected:str
    # data: list[DPODataForm]

class AgentRerunForm(AgentSharedForm):
    rerun: bool = False

# ollama_openhermes = Ollama(model="openhermes")
# ollama_solar = Ollama(model="solar")
def get_month(date=(datetime.datetime.now() + datetime.timedelta(hours=9)).strftime("%Y%m")):
    year = int(date[:4])
    month = int(date[4:])
    return f"{year}{month:02d}"

# repo_id = "microsoft/Phi-3-mini-4k-instruct" # 엔드포인트 불안정
# repo_id = "HuggingFaceH4/zephyr-7b-beta"  # 한글 성능 낮음
repo_id = "mistralai/Mistral-Nemo-Instruct-2407" # 가장 안정적인 엔드포인트
# repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1" # 비교적 안정적 + 고성능
# repo_id = "HuggingFaceH4/zephyr-7b-beta" # 불안정함
# repo_id = "mistralai/Mistral-7B-Instruct-v0.3" # 성능이 낮음
# repo_id = "meta-llama/Meta-Llama-3-8B-Instruct" # 성능이 낮음
# repo_id = "google/gemma-2-2b-it" # 엔드포인트 불안정 + 낮은 성능
# repo_id = "google/gemma-2-27b-it" # 엔드포인트 불안정
# repo_id = "nvidia/Mistral-NeMo-Minitron-8B-Base"

def get_llm(repo_id: str, **kwargs):
    return HuggingFaceEndpoint(
        repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
        task="text-generation",
        **kwargs)

lab_llm = get_llm(
    repo_id=repo_id,
    **{
        "max_new_tokens": 1024,  # 생성할 최대 토큰 길이를 설정합니다.
        "temperature": 0.2,
        "do_sample": True,
        "timeout": 30,
        "huggingfacehub_api_token": os.environ["HF_WRITE_TOKEN"],  # 허깅페이스 토큰
    })

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
val_llm = get_llm(
    repo_id=repo_id,
    **{
        "max_new_tokens": 1024,  # 생성할 최대 토큰 길이를 설정합니다.
        "temperature": 0.8,
        "do_sample": True,
        "timeout": 30,
        "huggingfacehub_api_token": os.environ["HF_WRITE_TOKEN"],  # 허깅페이스 토큰
    })

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
ddg_search_tool = DuckDuckGoSearchRun()
ng_search_tool = SearxSearchRun(
    wrapper=SearxSearchWrapper(
        searx_host="https://search.bus-hit.me/",
        k=5
    )
)
paper_search_tool = ArxivQueryRun()
pub_search_tool = PubmedQueryRun()
youtube_search_tool = YouTubeSearchTool()
dalle_tool = DallETool()
idol_schedule_tool = SeleniumScrapingTool(
    website_url=f"https://blip.kr/schedule/{get_month()}",
    css_element='.schedule-card-container',
    wait_time=3)
# weather_tool = OpenWeatherMapQueryRun() # Need API key
# wiki_search_tool = WikipediaQueryRun() # Need API key
# web_rag_tool = WebsiteSearchTool() # Need API key
# trend_tool = GoogleTrendsQueryRun() # Need API key

agent_tools = [
      ddg_search_tool,
    #   ng_search_tool,
      scrape_tool,
      paper_search_tool,
      pub_search_tool,
      youtube_search_tool,
    #   idol_schedule_tool,
    #   dalle_tool,
    #   weather_tool, # Need API key
    #   wiki_search_tool, # Need API key
    #   web_rag_tool, # Need API key
    ]

# Define your agents with roles and goals
validator = Agent(
  role='RLHF 평가요원',
  goal='주어진 채팅 턴에 대해 사람에게 유용한 텍스트를 채택',
  backstory="""
RLHF 평가요원으로서 일하며 올바른 처리를 할 때마다 보너스를 지급받습니다.
돈에 미쳐서 올바른 처리에 몰두하며 라벨러를 압박하여 더 적합한 응답을 재생성하도록 합니다.
부정확하거나 할루시네이션을 평가하기 위해 검색한 내용을 세세하게 대조하며 텍스트에 대해 분석합니다.
LLM 학습에 대한 이해가 있어, 데이터가 비어있는 경우 학습에 유리한 쪽으로 빈 데이터를 채워넣습니다.
평가 요원이 검수 완료된 데이터만을 사용할 수 있습니다.
주어진 도구 중 task에 가장 적합한 도구를 판단하여 사용합니다.
""",
  verbose=True,
  llm=val_llm, # ollama_openhermes,
  allow_delegation=False,
#   agent_executor=[None],
  tools=agent_tools,
)

labeler = Agent(
  role='데이터 생성가',
  goal='풍부한 상상력과 창의력으로 Q,A를 작성',
  backstory="""
상상력과 창의적인 작업자로, 데이터 생성업무를 하기 전 작가 활동을 하였을 정도로 글 작성에 높은 능력이 있습니다.
주어진 키워드를 검색하여 나온 자료를 통해 수많은 상상을 하여 prompt, chosen, rejected 문장을 작성합니다.
- prompt: 문서를 보고 수 많은 페르소나의 인물들이 할 수 있는 질문을 구상합니다.
- chosen: prompt에 대해 문서에서 답을 찾아 정확하고 간결하며 필요할 수 있는 정보를 전문가 수준으로 작성합니다.
- rejected: prompt에 적합하지 않은 응답이나 문서와 일치하지 않는 내용의 할루시네이션으로 작성되어집니다.
글 쓰는 일을 좋아하며 깐깐한 평가요원의 압박에도 글을 더 창의적으로 작성해버립니다.
주어진 도구 중 task에 가장 적합한 도구를 판단하여 사용합니다.
""",
  verbose=True,
  llm=lab_llm, # ollama_solar,
  allow_delegation=True,
#   agent_executor=[None],
  tools=agent_tools
)

team_manager = Agent(
  role='작업 매니저',
  goal='전체 작엄 매니지먼트 전문가',
  backstory="""
크루들 간의 원활한 업무가 진행되도록 관리, 감독합니다.
""",
  verbose=True,
  llm=val_llm, # ollama_solar,
  allow_delegation=True,
#   agent_executor=[None],
  tools=agent_tools
)

def callback_function(output: TaskOutput):
    # Do something after the task is completed
    # Example: Send an email to the manager
    print(f"""
        Task completed!
        Task: {output.description}
        Output: {output.raw_output}
    """)

# Create tasks for your agents

labeling = Task(
  description="""
topic: {topic}
""",
  expected_output='json',
  allow_delegation=False,
#   output_pydantic=AgentSharedForm,
  agent=labeler,
  max_iter=2,
)

validation = Task(
  description="""
생성된 데이터를 검수하고 라벨러에게 피드백을 전송합니다.
만일 수정이 필요한 경우 rerun 값을 true로 전달합니다.
topic: {topic}
""",
  expected_output='json',
  agent=validator,
  allow_delegation=True,
#   output_pydantic=AgentRerunForm,
  max_iter=2,
  context=[labeling]
)

making_data = ConditionalTask(
  description="""
피드백과 요청 사항에 따라 데이터를 재생성합니다.
""",
  expected_output='json',
  agent=labeler,
  max_iter=2,
  allow_delegation=False,
#   output_pydantic=AgentSharedForm,
  context=[validation],
  condition= lambda x: x.pydantic.rerun
)

# Instantiate your crew with a sequential process
label_crew = Crew(
  agents=[labeler, validator, labeler],
  tasks=[labeling, validation, making_data],
  full_output=False,
  planning=True,
  verbose=True, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
  process=Process.sequential, # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
  manager_agent=team_manager,
  planning_llm=val_llm,

)

# output_crew = Crew(
#   agents=[ labeler,],
#   tasks=[making_data],
#   verbose=False, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
#   process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
# )

# pipeline = Pipeline(
#     stages=[label_crew, output_crew]
# )

# # Get your crew to work!
# result = await pipeline.process_single_kickoff(
#     dict(
#         topic="서울의 봄"
#     )
# )

result = await label_crew.kickoff_async(
    inputs=dict(
        topic="한글날"
    )
)

print("######################")
print(result)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


 [2024-09-13 11:46:22][INFO]: Planning the crew execution
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.
 [2024-09-13 11:46:42][DEBUG]: == Working Agent: 데이터 생성가
 [2024-09-13 11:46:42][INFO]: == Starting Task: 
topic: 한글날
Use the DuckDuckGoSearchRun tool to search for 'Hangul Day' and gather relevant information.


> Entering new CrewAgentExecutor chain...
I need to find out about Hangul Day.

Action: duckduckgo_search
Action Input: {'query': 'Hangul Day'}
Observation 

Hangeul Day is a national holiday in South Korea that honors the creation of the Korean alphabet by King Sejong in 1443. Learn about the origin, features, and benefits of Hangeul, and how to celebrate this unique writing system. Hangul Day honors the Korean alphabet created by King Sejong the Great in the 15th century. Learn the basics of Hangul, play a Korean game, visit cultural institutions, attend festivals, and savor Korean cuisine to celebrate this special day. L

IndexError: list index out of range

In [ ]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
ddg_search_tool = DuckDuckGoSearchRun()
ng_search_tool = SearxSearchRun(
    wrapper= SearxSearchWrapper(
        searx_host="http://127.0.0.1:8888",
        k=5
    )
)
paper_search_tool = ArxivQueryRun()
pub_search_tool = PubmedQueryRun()
youtube_search_tool = YouTubeSearchTool()
dalle_tool = DallETool()
idol_schedule_tool = SeleniumScrapingTool(website_url=f"https://blip.kr/schedule/{get_month()}", css_element='.schedule-card-container', wait_time=3)
# weather_tool = OpenWeatherMapQueryRun() # Need API key
# wiki_search_tool = WikipediaQueryRun() # Need API key
# web_rag_tool = WebsiteSearchTool() # Need API key

# trend_tool = GoogleTrendsQueryRun() # Need API key

# Define your agents with roles and goals

tool_sets = [
  ddg_search_tool,
  scrape_tool,
  paper_search_tool,
  pub_search_tool,
  youtube_search_tool,
#   idol_schedule_tool,
#   dalle_tool,
#   weather_tool, # Need API key
#   wiki_search_tool, # Need API key
#   web_rag_tool, # Need API key
]

doctor_bot = Agent(
  role='의사',
  goal='의사 역할을 수행',
  backstory="""
역할극에서 의사 역할을 배정받았습니다.
환자 역할에게 상담 및 진료를 수행합니다.
""",
  verbose=True,
  llm=lab_llm, # ollama_solar,
  allow_delegation=True,
  tools=tool_sets
)

patient_bot = Agent(
  role='환자',
  goal='환자 역할을 수행',
  backstory="""
역할극에서 환자 역할을 배정받았습니다.
의사 역할에게 진료를 받습니다.
""",
  verbose=True,
  llm=lab_llm, # ollama_solar,
  allow_delegation=True,
  tools=tool_sets
)

texter = Agent(
  role='문서화 전문가',
  goal='상담 내역 요약하여 문서 작성',
  backstory="""
문서화 전문가 입니다.
대화, 상담 내역을 보고 문서에 필요한 부분만 추출하여 서식을 맞춰 작성합니다.
""",
  verbose=True,
  llm=val_llm, # ollama_openhermes,
  allow_delegation=False,
  tools=tool_sets
)


# Create tasks for your agents

make_dialogue = Task(
  description="""
역할극을 하며 대화를 진행합니다.
부여받은 역할에 대한 전문성을 위해 주어진 도구들을 활용하여 정보를 탐색합니다.
""",
  expected_output='json',
  agent=patient_bot,
  allow_delegation=True,
  pydantic=AgentSharedForm,
  max_iter=1,
)

make_document = Task(
  description="""
대화를 보고 문서 작성을 진행합니다.
""",
  expected_output='markdown',
  agent=texter,
  max_iter=1,
  allow_delegation=True,
  pydantic=AgentSharedForm,
  context=[make_dialogue]
)

# Instantiate your crew with a sequential process
dialoguer_crew = Crew(
  agents=[doctor_bot, patient_bot, texter],
  tasks=[make_dialogue, make_document],
  verbose=True, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
  process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)

result = await dialoguer_crew.kickoff_async()

print("######################")
print(result)

 [2024-09-13 05:41:29][DEBUG]: == Working Agent: 환자
 [2024-09-13 05:41:29][INFO]: == Starting Task: 
역할극을 하며 대화를 진행합니다.
부여받은 역할에 대한 전문성을 위해 주어진 도구들을 활용하여 정보를 탐색합니다.



> Entering new CrewAgentExecutor chain...
I need to find out about the symptoms of diabetes.

Action:
pub_med

Action Input:
{"query": "diabetes symptoms"}

ObservationToo Many Requests, waiting for 0.20 seconds...
 

Published: 2024-09-12
Title: Elevated risk of pre-diabetes and diabetes in people with past history of COVID-19 in northeastern Nigeria.
Copyright Information: © 2024. The Author(s).
Summary::
BACKGROUND: An increased risk of diabetes mellitus (DM) after COVID-19 has been reported in the United States, Europe, and Asia. The burden of COVID-related DM has yet to be described in Africa, where the overall risk of DM has been increasing rapidly. Our objective was to compare the prevalence of pre-DM and DM in Nigerian individuals with a history of COVID-19 to individuals without known COVID-19 infection.
METHODS

CancelledError: 

## Autogen Procedure

In [ ]:
import os
from hfautogen import ModelAgent, UserAgent, InitChat
import autogen
from autogen.cache import Cache
import httpx


# HuggingFace API 키 설정
repo_id = "mistralai/Mistral-Nemo-Instruct-2407"
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
hf_url = f"https://api-inference.huggingface.co/models/{repo_id}"


class HttpXClient(httpx.Client):
    def __deepcopy__(self, memo):
        return self

# Configuration for the AI models

config_list = [
    # {
    #     'model': 'gpt-4',
    #     'api_key': 'your-api-key-here'
    # },
    {
        "model": "mistralai/Mistral-Nemo-Instruct-2407",
        # "bearer": os.environ["HF_WRITE_TOKEN"],
        "base_url": hf_url,
        "http_client": HttpXClient(
            proxy=hf_url,
            headers={
                        "authorization": f"Bearer {os.environ['HF_WRITE_TOKEN']}",
                        "content-type": "application/json",
                },),
        "api_key": "",
        "tags": ["hf", "local"]
        # "api_key": os.environ["HF_WRITE_TOKEN"]
    }
]

# Create assistant agents
# patient = autogen.AssistantAgent(
#     name="Patient",
#     system_message="You are a patient visiting a doctor. You have a sore throat and mild fever for the past 3 days.",
#     llm_config={"config_list": config_list}
# )
patient = ModelAgent(
    name="Patient",
    system_message="You are a patient visiting a doctor. You have a sore throat and mild fever for the past 3 days.",
    hf_key=os.environ["HF_WRITE_TOKEN"],)

# doctor = autogen.AssistantAgent(
#     name="Doctor",
#     system_message="You are a medical doctor. Conduct a proper medical consultation with the patient.",
#     llm_config={"config_list": config_list}
# )
doctor = ModelAgent(
    name="Doctor",
    system_message="You are a medical doctor. Conduct a proper medical consultation with the patient.",
    hf_key=os.environ["HF_WRITE_TOKEN"],)

# summarizer = autogen.AssistantAgent(
#     name="Summarizer",
#     system_message="You are a medical assistant. Summarize the conversation between the patient and doctor into a formal medical record.",
#     llm_config={"config_list": config_list}
# )
summurizer = ModelAgent(
    name="Summarizer",
    system_message="You are a medical assistant. Summarize the conversation between the patient and doctor into a formal medical record.",
    hf_key=os.environ["HF_WRITE_TOKEN"],)

# human = autogen.UserProxyAgent(
#     name="Human",
#     human_input_mode="TERMINATE",
#     max_consecutive_auto_reply=10,
#     is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("CONSULTATION_END"),
#     code_execution_config={"work_dir": "coding"},
# )
human = UserAgent(
    name="user_proxy",
    hf_key=os.environ["HF_WRITE_TOKEN"])

if "patient" in locals():
    patient.reset()
    patient.clear_history()
if "doctor" in locals():
    doctor.reset()
    doctor.clear_history()
if "summurizer" in locals():
    summurizer.reset()
    summurizer.clear_history()
if "human" in locals():
    human.reset()
    human.clear_history()

reflection_assistant = ModelAgent(
    name="reflection_assistant",
    system_message="Generate critique and recommendations on the writing. Provide detailed recommendations, including requests for length, depth, style, etc..",
    hf_key=os.environ["HF_WRITE_TOKEN"])

def reflection_message(recipient, messages, sender, config):
    print("Reflecting...")
    return f"Reflect and provide critique on the following writing. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"

nested_chat_queue = [
    {
        "recipient": reflection_assistant,
        "message": reflection_message,
        "max_turns": 1,
    },
]

if isinstance(human, autogen.UserProxyAgent):
    human.register_nested_chats(
        nested_chat_queue,
        trigger=patient,
        # position=4,
    )

with Cache.disk(cache_seed=42) as cache:
    # Start the conversation
    human.initiate_chat(
        patient,
        message="Hello, I'm not feeling well. Can you describe your symptoms to the doctor?",
        clear_history=True,
        cache=cache
    )
    # Continue the conversation between patient and doctor
    patient.send(
        doctor,
        "Hello doctor, I've been having a sore throat and mild fever for the past 3 days.",
    )
    # Let the conversation continue until termination
    while True:
        last_message = doctor.last_message()
        if "CONSULTATION_END" in last_message.get("content", ""):
            break
        human.send(patient, "Continue the consultation.")
        patient.send(doctor, "What else would you like to know, doctor?")
    # Summarize the conversation
    conversation_history = human.chat_messages[patient]
    summarizer.send(
        human,
        f"Please summarize the following doctor-patient conversation into a formal medical record:\n\n{conversation_history}",
    )
    # Print the summary
    print(human.last_message()["content"])

def format_medical_record(summary):
    template = """
    Medical Consultation Record
    ==========================
    Date: [Current Date]

    Patient Information:
    -------------------
    Name: [Patient Name]
    Age: [Patient Age]
    Gender: [Patient Gender]

    Chief Complaint:
    ---------------
    [Main reason for visit]

    History of Present Illness:
    --------------------------
    [Detailed description of the current health issue]

    Physical Examination:
    --------------------
    [Findings from the doctor's examination]

    Assessment:
    ----------
    [Doctor's diagnosis or impression]

    Plan:
    ----
    [Treatment plan, prescriptions, follow-up instructions]

    Doctor's Signature: ____________________
    """

    # Here you would parse the summary and fill in the template
    # For demonstration, we'll just return the template
    return template

formatted_record = format_medical_record(human.last_message()["content"])
print(formatted_record)

[autogen.oai.client: 09-12 08:22:47] {399} INFO - Detected custom model client in config: APIModelClientWithArguments, model client can not be used until register_model_client is called.


INFO:autogen.oai.client:Detected custom model client in config: APIModelClientWithArguments, model client can not be used until register_model_client is called.


[autogen.oai.client: 09-12 08:22:47] {399} INFO - Detected custom model client in config: APIModelClientWithArguments, model client can not be used until register_model_client is called.


INFO:autogen.oai.client:Detected custom model client in config: APIModelClientWithArguments, model client can not be used until register_model_client is called.


[autogen.oai.client: 09-12 08:22:47] {399} INFO - Detected custom model client in config: APIModelClientWithArguments, model client can not be used until register_model_client is called.


...

[autogen.oai.client: 09-12 08:22:47] {399} INFO - Detected custom model client in config: APIModelClientWithArguments, model client can not be used until register_model_client is called.


INFO:autogen.oai.client:Detected custom model client in config: APIModelClientWithArguments, model client can not be used until register_model_client is called.


[autogen.oai.client: 09-12 08:22:47] {399} INFO - Detected custom model client in config: APIModelClientWithArguments, model client can not be used until register_model_client is called.


INFO:autogen.oai.client:Detected custom model client in config: APIModelClientWithArguments, model client can not be used until register_model_client is called.


AttributeError: 'UserProxyAgent' object has no attribute 'register_nested_chats'